In [50]:
import seaborn as sb
import os
import time

import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#import os
#from itertools import combinations
#import random
import osmnx as ox
from sklearn.preprocessing import MinMaxScaler

In [2]:
names = ["Visano", "Asola","Montichiari","Brescia","Roma"]
places = [(45.317468,10.3694991), # Visano (BS)
        (45.221332657542824, 10.41757427434648), # Asola (MN)
        (45.41301145796791, 10.39169903519818), # Montichiari (BS)
        (45.536282844234854, 10.206297121183406), # Brescia (BS)
        (41.90343248728792, 12.496484448996995)] # Roma (RO)

## Download maps and save graph as .xml

In [3]:
for c in places:
    name = names[places.index(c)]
    print(name)
    graph = ox.graph_from_point(c,dist=3000,network_type="drive")
    # conversions
    graph = nx.convert_node_labels_to_integers(graph)
    graph = graph.to_undirected()
    ox.save_graphml(graph,filepath='data/'+name+'_graph.xml')
    
    #ox.plot_graph(graph, node_size=2, figsize=(18,18))
 
    print(graph)
    

Visano
MultiGraph with 297 nodes and 378 edges
Asola
MultiGraph with 480 nodes and 664 edges
Montichiari
MultiGraph with 1130 nodes and 1509 edges
Brescia
MultiGraph with 3925 nodes and 5464 edges
Roma
MultiGraph with 4729 nodes and 7277 edges


## Load graphs

In [3]:
for name in names:
    
    filename = 'data/' + name + '_graph.xml'
    g = ox.load_graphml(filepath=filename)
    print(name)
    print(g)
    
    #ox.plot_graph(g, node_size=2, figsize=(18,18))

Visano
MultiGraph with 297 nodes and 378 edges
Asola
MultiGraph with 480 nodes and 664 edges
Montichiari
MultiGraph with 1130 nodes and 1509 edges
Brescia
MultiGraph with 3925 nodes and 5464 edges
Roma
MultiGraph with 4729 nodes and 7277 edges


## Dijkstra

Definizione della funzione per la creazione della stringa nella formattazione corretta per la lettura da parte di Minizinc

In [4]:
def dataStringGenerator(df, nome_variabile):
    stringa = nome_variabile +" = [";
    ## sezione per la matrice delle distanze
    if df.shape[0] == df.shape[1]:
        for i in range(df.shape[0]):
            if i == 0:
                stringa = stringa + "| "
            else:
                stringa = stringa + "         | "
            for j in range(df.shape[1]):
                stringa = stringa + str(df.iat[i, j]) 
                if j != len(df)-1:
                    stringa = stringa + ", "
                else:
                    stringa = stringa +  os.linesep # aggiunta di \n a fine della riga 

        stringa = stringa + "         |];"
    # sezione per i semplici array di numeri
    else:
        for i in range(df.shape[0]):
            if i != df.shape[0]-1:
                stringa = stringa + str(df.iat[i,0]) + ", "
            else:
                stringa = stringa + str(df.iat[i,0])
        
        stringa = stringa + "];"
    
    return stringa

In [22]:
mDists = []

for name in ['Visano']:#names:
    
    print(name)
    
    filename = 'data/' + name + '_graph.xml'
    g = ox.load_graphml(filepath=filename)
    
    
    labels = nx.get_edge_attributes(g, 'length')
    dictDist = dict(nx.all_pairs_dijkstra_path_length(g,weight='length'))
    mDist = pd.DataFrame(dictDist)
    #mDists.append(mDist)
    mDistSorted = mDist.sort_index()
    
    # Scale all distances between 0..14
    scaler = MinMaxScaler(feature_range=(0, 7))
    mDistSortedScaled = scaler.fit_transform(mDistSorted)
    
    string = dataStringGenerator(pd.DataFrame(mDistSortedScaled ,dtype=int), "mdist")
    
    mDistSortedScaled
    #mDistSortedScaled.to_csv('data/' + name + '_dijkstra.csv')
    
    #filename2write = 'data/' + name + '_dijkstra.dzn'
    #file = open(filename2write, 'w')
    #file.write(string + os.linesep)
    #file.close()

    #print(string)
    
    pd.DataFrame(mDistSortedScaled).to_csv('data/' + name + '_dijkstra.csv', index=False)


Visano


/tmp/ipykernel_100768/4002206819.py:21: FutureWarning: In a future version, passing float-dtype values and an integer dtype to DataFrame will retain floating dtype if they cannot be cast losslessly (matching Series behavior). To retain the old behavior, use DataFrame(data).astype(dtype)
  string = dataStringGenerator(pd.DataFrame(mDistSortedScaled ,dtype=int), "mdist")


In [23]:
df = pd.read_csv('data/Visano_dijkstra.csv',index_col=False)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,287,288,289,290,291,292,293,294,295,296
0,0.000000,1.253784,0.885569,0.858802,0.846488,0.832392,0.484368,0.464897,0.454539,0.426012,...,6.302817,6.319487,6.145857,6.033958,4.788728,4.828391,0.896345,0.546537,0.594228,0.688361
1,1.161266,0.000000,0.312281,0.335032,0.345475,0.357430,0.652539,0.669565,0.678350,0.704047,...,5.166151,5.209999,5.187789,5.220456,3.514110,3.576636,1.884462,1.283606,1.325850,1.432781
2,0.858522,0.326863,0.000000,0.023813,0.034744,0.047257,0.356146,0.373968,0.383162,0.409596,...,5.706810,5.737731,5.722068,5.751001,3.846405,3.902970,1.546517,0.965862,1.010454,1.111868
3,0.835414,0.351874,0.023895,0.000000,0.010968,0.023524,0.333522,0.351405,0.360631,0.387121,...,5.748179,5.778111,5.762950,5.791597,3.871831,3.927940,1.520722,0.941609,0.986380,1.087373
4,0.824728,0.363411,0.034917,0.010985,0.000000,0.012576,0.323061,0.340971,0.350211,0.376728,...,5.767263,5.796738,5.781808,5.810323,3.883560,3.939459,1.508794,0.930393,0.975247,1.076046


In [56]:
dest = pd.DataFrame([256,25,100,2])


dest_sort = np.array(dest)
dest_sort = np.append(dest_sort, 0)
dest_sort = np.sort(dest_sort)

print([d  for d in dest[0]])
#print(dest_sort)

df_r = df[[str(d) for d in dest]]
df_r = df_r.loc[dest]
df_r = df_r.sort_index()
df_r = df_r.astype(int)


[256, 25, 100, 2]


ValueError: Cannot index with multidimensional key

In [42]:
string = dataStringGenerator(df_r, "mdist")
print(string)

mdist = [| 0, 0, 6, 4, 0
         | 0, 0, 6, 3, 0
         | 5, 5, 0, 5, 5
         | 4, 3, 5, 0, 4
         | 0, 0, 6, 4, 0
         |];


In [ ]:
[d. for d in dest]

In [ ]:
# Remember: when reading model output the index will start from 1